In [ ]:
from src.database.base import init_db, get_db_session
from src.utils.config import settings

from src.ingestion.ingestion_helpers import ingest_financial_data

from src.ingestion.edgar_db.accessors import edgar_login, get_10k_filing, get_company
import matplotlib.pyplot as plt

from src.database.companies import list_all_companies
from src.database.filings import Filing
from src.database.financial_concepts import FinancialConcept
import pandas as pd
from src.database.filings import get_filings_by_company
from src.database.companies import get_company_by_ticker
from src.database.financial_values import FinancialValue
init_db(settings.database.url)
session = get_db_session()


In [ ]:


# Query all financial concepts and convert to DataFrame
financial_concepts = session.query(FinancialConcept).all()

# Convert to list of dictionaries
concepts_data = []
for concept in financial_concepts:
    concept_dict = {column.name: getattr(concept, column.name) for column in concept.__table__.columns}
    concepts_data.append(concept_dict)

# Create DataFrame
df = pd.DataFrame(concepts_data)

In [ ]:


# Get Microsoft company
msft_company = get_company_by_ticker('MSFT')
if msft_company:
    print(f"Found Microsoft: {msft_company}")

    # Get all filings for Microsoft
    msft_filings = get_filings_by_company(msft_company.id)

    # Filter for 2024 10-K filings
    msft_2024_10k = [filing for filing in msft_filings
                     if filing.filing_type == '10-K' and filing.filing_date.year == 2024]

    if msft_2024_10k:
        filing = msft_2024_10k[0]  # Get the first (likely only) 2024 10-K
        print(f"Found 2024 10-K filing: {filing}")
        print(f"Filing Date: {filing.filing_date}")
        print(f"Accession Number: {filing.accession_number}")
        print(f"Period of Report: {filing.period_of_report}")
    else:
        print("No 2024 10-K filing found for Microsoft")
        # Show available filings for context
        print(f"Available filings: {len(msft_filings)}")
        for filing in msft_filings[:5]:  # Show first 5 filings
            print(f"  {filing.filing_type} - {filing.filing_date}")
else:
    print("Microsoft (MSFT) not found in database")

In [ ]:


# Query all financial values for the Microsoft 2024 10-K filing
financial_values = session.query(FinancialValue).filter(
    FinancialValue.filing_id == filing.id
).join(FinancialConcept).all()

# Convert to list of dictionaries with concept name, description, and value
values_data = []
for fv in financial_values:
    values_data.append({
        'concept_name': fv.concept.name,
        'description': fv.concept.description,
        'value': fv.value
    })

# Create DataFrame
values_df = pd.DataFrame(values_data)
print(f"Found {len(values_df)} financial values for Microsoft 2024 10-K filing")
values_df.head(10)

In [ ]:


company = get_company_by_ticker("GOOGL")
all_filings = get_filings_by_company(company.id)


# Get all financial values with their concepts and filing dates
time_series_data = []
for filing in all_filings:
    filing_year = filing.filing_date.year
    financial_values = session.query(FinancialValue).filter(
        FinancialValue.filing_id == filing.id
    ).join(FinancialConcept).all()

    for fv in financial_values:
        time_series_data.append({
            'concept_name': fv.concept.name,
            'description': fv.concept.description,
            'value': fv.value,
            'filing_year': filing_year,
            'filing_date': filing.filing_date
        })

# Create DataFrame
time_series_df = pd.DataFrame(time_series_data)

# Pivot to get years as columns
if not time_series_df.empty:
    pivot_df = time_series_df.pivot_table(
        index=['concept_name', 'description'],
        columns='filing_year',
        values='value',
        aggfunc='first'
    ).reset_index()

    # Clean up column names
    pivot_df.columns.name = None
    year_columns = [col for col in pivot_df.columns if isinstance(col, int)]
    other_columns = [col for col in pivot_df.columns if not isinstance(col, int)]

    print(f"Found financial data across {len(year_columns)} years: {sorted(year_columns)}")
    print(f"Total concepts: {len(pivot_df)}")
    pivot_df.head()


In [ ]:
pivot_df

In [ ]:

# Get the first 3 concepts from pivot_df
first_3_concepts = pivot_df[pivot_df['concept_name'].str.startswith('us-gaap')].head(3)

# Set up the plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot each concept
for i, (idx, row) in enumerate(first_3_concepts.iterrows()):
    concept_name = row['concept_name']
    description = row['description']

    # Get year columns and their values
    years = [2021, 2022, 2023, 2024]
    values = [float(row[year]) if pd.notna(row[year]) else 0 for year in years]

    # Create the plot
    axes[i].plot(years, values, marker='o', linewidth=2, markersize=6)
    axes[i].set_title(f"{description}\n({concept_name})", fontsize=10)
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Value')
    axes[i].grid(True, alpha=0.3)

    # Format y-axis to show values in billions
    axes[i].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e9:.1f}B'))

plt.tight_layout()
plt.show()